#### Unfixed bugs:  
- If i delete file, git won't commit that. File in proxy directory will be untouched  
- Tables are freaken  
- Directories must be added automaticly. Not in white list in code

In [1]:
import os, re, distutils.dir_util

In [2]:
def replace_carriages(filename, path):
    with open(os.path.join(path, filename), 'r', encoding='utf-8') as f:
        text = f.read()
        text = text.replace('  \n', '\n')
        text = text.replace('\n', '  \n')
        re.search(r'!\[\[\w.+\]\]', text)
        f.close()
    with open(os.path.join(path, filename), 'w', encoding='utf-8') as f:
        f.write(text)

In [3]:
def fix_images(filename, path, gitpath):
    with open(os.path.join(path, filename), 'r', encoding='utf-8') as f:
        text = f.read() 
        pattern = r'!\[\[\w.+\]\]'
        counter = 0
        while True:
            match = re.search(pattern, text)
            if not match:
                break
            span = match.span()
            img = text[span[0]:span[1]][3:-2]
            if '|' in img:
                name, size = img.split('|')
                raw_name = name
                name = name.replace(' ', '%20')
                new_img = f'![{raw_name}|{size}](https://github.com/PolkaDott/Data-Science-Summaries/blob/main/Язык%20R/attachments/{name}?raw=true)'
            else:
                name = img
                fixed_name = name.replace(' ', '%20')
                fixed_path = path.replace(' ', '%20')
                new_img = f'![{name}](https://github.com/PolkaDott/Data-Science-Summaries/blob/main/{fixed_path}/attachments/{fixed_name}?raw=true)'
            text = re.sub(pattern, new_img, text, count=1)
    if not os.path.isdir(os.path.join(gitpath, path)):
        os.makedirs(os.path.join(gitpath, path))
    with open(os.path.join(gitpath, path, filename), 'w', encoding='utf-8') as f:
        f.write(text)

In [4]:
def get_gitpath():
    path = os.getcwd()
    #path = os.path.normpath(path)
    objs = path.split(os.sep)
    dirname = objs[-1]
    objs[-1] = dirname+'ForGit'
    fullpath = os.path.join(*objs) # C:Users ...
    gitpath = os.path.join('..', objs[-1])
    if  not os.path.isdir(gitpath):
        raise Exception('Does not exist ' + gitpath + '. Rename or create it! And do not forget paste .git there')
    return gitpath

### get all .md files

In [6]:
dirs = os.listdir()
white_list_dirs = ['Мат.Статистика', 'Язык R', 'Прочее', 'Python в data science']
dirs = list(filter(lambda x: x in white_list_dirs, dirs))
files = {dir : [] for dir in dirs}
for dir in dirs:
    mds = list(filter(lambda x: x.endswith('.md'), os.listdir(dir)))
    # mds = list(map(lambda x: dir+'/'+x, mds))
    files[dir] = files[dir] + mds
print(files)

{'Python в data science': ['Функции.md'], 'Мат.Статистика': ['Дисперсионный анализ несвязных выборок.md', 'Корреляция.md', 'Основы.md', 'Регрессия и регрессионный анализ.md', 'Сравнение двух несвязных выборок.md', 'Сравнение двух связных выборок.md'], 'Прочее': ['Пробелы.md'], 'Язык R': ['ANOVA.md', 'Графики.md', 'Регрессионный анализ.md', 'Типы данных.md', 'Функции.md', 'Хитрости языка.md']}


### Fix carriages to view in git properly

In [7]:
for dir in files:
    for file in files[dir]:
        replace_carriages(file, dir)
print('done')

done


### Create new path near current directory to push it to github. It will be fixed images to view them in github in web

In [8]:
for dir in files:
    for file in files[dir]:
        fix_images(file, dir, get_gitpath())
print('done')

done


### In order to push in git i have to be sure images are in their directories

In [9]:
for dir in files:
    distutils.dir_util.copy_tree(os.path.join(dir, 'attachments'), os.path.join(get_gitpath(), dir, 'attachments'))
print('done')

done


### I have to send new version of this script

In [10]:
fixer_name = 'fix_md_to_git.ipynb'
fixer = open(fixer_name, 'r', encoding='utf-8')
update_fixer = open(os.path.join(get_gitpath(), fixer_name), 'w', encoding='utf-8')
update_fixer.write(fixer.read())
update_fixer.close()
fixer.close()